In [1]:
from collections import namedtuple
import time
import warnings

from kmodes.kmodes import KModes
from kmodes.util import dissim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm


In [2]:
Record = namedtuple(
    "Record",
    [
        "n_clusters",
        "initialisation",
        "seed",
        "initial_cost",
        "final_cost",
        "n_iterations",
        "time",
    ],
)


def find_clustering(data, n_clusters, initialisation, seed):

    start = time.perf_counter()
    km = KModes(n_clusters, init=initialisation, n_init=1, random_state=seed)
    km.fit(data)
    end = time.perf_counter()

    return km.epoch_costs_[0], km.cost_, km.n_iter_, end - start


def run_experiment(dataset, initialisation, repetitions):

    data = dataset.drop("class", axis=1)
    n_clusters = dataset["class"].nunique()

    results = []
    for seed in tqdm.tqdm(range(repetitions)):
        initial_cost, final_cost, n_iter, time = find_clustering(
            data, n_clusters, initialisation, seed
        )

        record = Record(
            n_clusters, initialisation, seed, initial_cost, final_cost, n_iter, time
        )
        results.append(record)

    return pd.DataFrame(results)


def main(name, repetitions=250, root="../data/", destination=None):

    data = pd.read_csv(f"{root}{name}.csv", na_values=["?", "dna"])
    dataset = data.dropna()

    dfs = [
        run_experiment(dataset, initialisation, repetitions)
        for initialisation in ("cao", "huang", "matching")
    ]

    df = pd.concat(dfs, axis=0, ignore_index=True)
    if destination is not None:
        df.to_csv(destination + f"{name}_results.csv", index=False)

    return df


In [3]:
for name in ("breast_cancer", "mushroom", "soybean", "nursery"):
    main(name, root="../data/", destination="../data/nclasses/")


100%|██████████| 250/250 [05:39<00:00,  1.36s/it]
